## Handle Key

In [11]:
import os
from dotenv import load_dotenv

load_dotenv()
api_id = os.getenv('api_id')
api_hash = os.getenv('api_hash')

## Retrieve from telegram

In [ ]:
import csv
import time
from datetime import datetime
from telethon import TelegramClient
from pytz import timezone

session_name = 'my_session'
channel_input = 'https://t.me/Bibitid'
wib_timezone = timezone('Asia/Jakarta')
start_date = wib_timezone.localize(datetime(2025, 5, 25))
end_date = wib_timezone.localize(datetime(2025, 5, 27))
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_filename = f'telegram_messages_{timestamp}.csv'
batch_size = 10000

async def main():
  async with TelegramClient(session_name, api_id, api_hash) as client:
    channel = await client.get_entity(channel_input)

    buffer = []
    total_count = 0

    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
      writer = csv.writer(file, quoting=csv.QUOTE_ALL)
      writer.writerow([
        'id',
        'date',
        'text',
        'sender_id',
        'chat_id',
        'reply_to_msg_id',
        'views',
        'forwards',
        'buttons',
        'raw_text',
        'message_link'
      ])

    async for msg in client.iter_messages(channel, offset_date=start_date, reverse=True):
      if msg.date > end_date:
        break

      row = [
        msg.id,
        msg.date.astimezone(wib_timezone).strftime('%a %b %d %H:%M:%S %z %Y') if msg.date else "",
        msg.text.replace('\n', ' ').strip() if msg.text else "",
        msg.sender_id if msg.sender_id else '',
        getattr(msg, 'chat_id', getattr(msg.to_id, 'channel_id', '')),
        msg.reply_to_msg_id if msg.reply_to_msg_id else '',
        msg.views if msg.views is not None else '',
        msg.forwards if msg.forwards is not None else '',
        len(msg.buttons) if msg.buttons else 0,
        msg.raw_text.replace('\n', ' ').strip() if msg.raw_text else '',
        f'https://t.me/{channel.username}/{msg.id}'
      ]

      buffer.append(row)
      total_count += 1

      if total_count % batch_size == 0:
        with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
          writer = csv.writer(file, quoting=csv.QUOTE_ALL)
          writer.writerows(buffer)

        buffer.clear()
        print(f'Written {total_count} messages. Sleeping for 10 seconds...')
        time.sleep(10)

    if buffer:
      with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_ALL)
        writer.writerows(buffer)
      print(f'Final batch written. Total messages: {total_count}')

    print(f'Done! Messages saved to {csv_filename}.')

await main()

## Move file for later processing

In [ ]:
file_path = './data/'
!mkdir -p $file_path
!mv $csv_filename $file_path
print(f'Successfully moved {csv_filename} to {file_path}')

## Retrieve stored telegram messages

In [ ]:
import os
import pandas as pd
from pandarallel import pandarallel

pandarallel.initialize()

dtype = {
  'id': 'string',
  'date': 'string',
  'text': 'string',
  'sender_id': 'string',
  'chat_id': 'string',
  'reply_to_msg_id': 'string',
  'views': 'Int64',
  'forwards': 'Int64',
  'buttons': 'Int64',
  'raw_text': 'string',
  'message_link': 'string'
}

try:
  filename = 'telegram_messages_20250527_151905.csv'
  file_path = f'./data/{filename}'

  if not os.path.exists(file_path):
    raise FileNotFoundError(f'File {file_path} does not exist')
  
  print(f'Reading from {file_path} to dataframe')

  original_df = pd.read_csv(file_path, dtype=dtype)
  original_df['date'] = pd.to_datetime(
    original_df['date'],
    format='%a %b %d %H:%M:%S %z %Y',
    errors='coerce'
  )
  original_df['date'] = original_df['date'].parallel_apply(
    lambda x: x.replace(tzinfo=None) if pd.notnull(x) else x
  )
  print(f'Successfully read from {file_path} to dataframe')

except FileNotFoundError:
  original_df = pd.DataFrame()
  print(f'File not found')

except Exception as e:
  original_df = pd.DataFrame()
  print(f'Error reading file: {e}')

display(original_df)

## Analyze data distribution

In [ ]:
from pandarallel import pandarallel

pandarallel.initialize()

monthly_counts = original_df.groupby(original_df['date'].dt.to_period('M')).size()
monthly_counts.index = monthly_counts.index.to_timestamp()

quarterly_counts = original_df.groupby(original_df['date'].dt.to_period('Q')).size()
quarterly_counts.index = quarterly_counts.index.to_timestamp()

def get_semester(date):
  if pd.isnull(date):
    return None
  return f'{date.year}-S1' if date.month <= 6 else f'{date.year}-S2'

original_df['semester'] = original_df['date'].parallel_apply(get_semester)
semesterly_counts = original_df.groupby('semester').size()

yearly_counts = original_df.groupby(original_df['date'].dt.to_period('Y')).size()
yearly_counts.index = yearly_counts.index.to_timestamp()

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# monthly_counts.sort_index().plot(label='Monthly')
quarterly_counts.sort_index().plot(label='Quarterly')
# semesterly_counts.sort_index().plot(label='Semesterly')
# yearly_counts.sort_index().plot(label='Yearly')

plt.title('Message Volume Over Time')
plt.xlabel('Time')
plt.ylabel('Number of Messages')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Construct threaded messages

In [ ]:
def build_merged_text(df):
  msg_dict = df.set_index('id').to_dict('index')
  merged_cache = {}

  def get_merged_text(msg_id):
    if msg_id in merged_cache:
      return merged_cache[msg_id]

    msg = msg_dict.get(msg_id)
    if not msg:
      return ''

    current_text = msg.get('text') or ''

    reply_id = msg.get('reply_to_msg_id')
    if pd.notna(reply_id):
      parent_text = get_merged_text(reply_id)
      merged = f'{parent_text} RT {current_text}'
    else:
      merged = current_text

    merged_cache[msg_id] = merged
    return merged

  df['merged_text'] = df['id'].apply(get_merged_text)
  return df

original_df = original_df.sort_values(by='date')

original_df = build_merged_text(original_df)

display(original_df)

## Preprocessing functions

In [4]:
import emoji
import re
import string
from unidecode import unidecode
from indoNLP.preprocessing import pipeline, remove_html, remove_url, replace_slang, replace_word_elongation

# Pre-compiled regex patterns
USERNAME_RE = re.compile(r'@\w+')
RT_RE = re.compile(r'\brt\b', flags=re.IGNORECASE)
HASHTAG_RE = re.compile(r'#')
DIGIT_RE = re.compile(r'\d+')
WHITESPACE_RE = re.compile(r'\s+')

# Pre-compiled translation
PUNCT_TRANSLATOR = str.maketrans('', '', string.punctuation)

def fast_clean(text):
  # Normalize Unicode (remove fancy fonts, underlines)
  text = unidecode(text)

  # Case folding to lowercase
  text = text.lower()

  # Remove usernames
  text = USERNAME_RE.sub('', text)

  # Remove RT
  text = RT_RE.sub('', text)

  # Remove hashtag symbol but keep the word
  text = HASHTAG_RE.sub('', text)

  # Remove digits
  text = DIGIT_RE.sub('', text)

  # Remove punctuation
  text = text.translate(PUNCT_TRANSLATOR)

  # Remove emojis
  text = emoji.replace_emoji(text, replace='')

  # Remove extra whitespace
  text = WHITESPACE_RE.sub(' ', text).strip()

  return text

## Run preprocessing

In [ ]:
from pandarallel import pandarallel

pandarallel.initialize()

preprocessed_df = original_df.copy()

preprocessed_df.dropna(subset=[
  'text',
  'raw_text'
], inplace=True)

preprocessed_df['fast_clean'] = preprocessed_df['merged_text'].parallel_apply(fast_clean)

indonlp_pipeline = pipeline([
  remove_html,
  remove_url,
  replace_slang,
  replace_word_elongation
])

preprocessed_df['basic_clean'] = preprocessed_df['fast_clean'].parallel_apply(indonlp_pipeline)

preprocessed_df.dropna(subset=[
  'text',
  'raw_text',
  'fast_clean',
  'basic_clean'
], inplace=True)

display(preprocessed_df)

## Filter out text that is shorter than 3 words

In [6]:
preprocessed_df = preprocessed_df[preprocessed_df['basic_clean'].str.split().str.len() > 3].copy()

display(preprocessed_df)

,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link,semester,merged_text,fast_clean,basic_clean
0,1057066,2022-03-13 10:25:11,"Kak, untuk penjelasan & informasi lebih lanjut...",5033891202,-1001360531894,1057060,<NA>,<NA>,0,"Kak, untuk penjelasan & informasi lebih lanjut...",https://t.me/Bibitid/1057066,2022-S1,"RT Kak, untuk penjelasan & informasi lebih la...",kak untuk penjelasan informasi lebih lanjut si...,kak untuk penjelasan informasi lebih lanjut si...
1,1057071,2022-03-13 10:28:17,sebelum tutup... dijual. pasti ada info lewat...,1368241084,-1001360531894,1057067,<NA>,<NA>,0,sebelum tutup... dijual. pasti ada info lewat...,https://t.me/Bibitid/1057071,2022-S1,RT sebelum tutup... dijual. pasti ada info l...,sebelum tutup dijual pasti ada info lewat emai...,sebelum tutup dijual pasti ada info lewat emai...
4,1057077,2022-03-13 10:29:51,Trenggalek itu di planet mana? 🌝,5191257541,-1001360531894,1057072,<NA>,<NA>,0,Trenggalek itu di planet mana? 🌝,https://t.me/Bibitid/1057077,2022-S1,RT Trenggalek itu di planet mana? 🌝,trenggalek itu di planet mana,trenggalek itu di planet mana
5,1057079,2022-03-13 10:30:04,"Nah, kalau lewat 3x masih nanya hal yang sama,...",5028290355,-1001360531894,1057073,<NA>,<NA>,0,"Nah, kalau lewat 3x masih nanya hal yang sama,...",https://t.me/Bibitid/1057079,2022-S1,"RT Nah, kalau lewat 3x masih nanya hal yang s...",nah kalau lewat x masih nanya hal yang sama sk...,nah kalau lewat kali masih bertanya hal yang s...
7,1057081,2022-03-13 10:30:24,Khoceng gembul 😍😍😍,1609430053,-1001360531894,1057077,<NA>,<NA>,0,Khoceng gembul 😍😍😍,https://t.me/Bibitid/1057081,2022-S1,RT Trenggalek itu di planet mana? 🌝 RT Khocen...,trenggalek itu di planet mana khoceng gembul,trenggalek itu di planet mana khoceng gembul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822470,2537812,2025-05-24 21:34:12,Kejadian pas bulan Maret yaa?,6819984040,-1001360531894,2537811,<NA>,<NA>,0,Kejadian pas bulan Maret yaa?,https://t.me/Bibitid/2537812,2025-S1,Saham yang cocok buat pemula gais🙏🏽 RT RT Kok...,saham yang cocok buat pemula gais kok bisa pad...,saham yang cocok buat pemula gais kok bisa pad...
822471,2537813,2025-05-24 21:34:49,Coba buka salah satu saham Dan klik grafik 3M ...,1194598784,-1001360531894,2537812,<NA>,<NA>,0,Coba buka salah satu saham Dan klik grafik 3M ...,https://t.me/Bibitid/2537813,2025-S1,Saham yang cocok buat pemula gais🙏🏽 RT RT Kok...,saham yang cocok buat pemula gais kok bisa pad...,saham yang cocok buat pemula gais kok bisa pad...
822474,2537821,2025-05-24 21:52:25,Terakhir tanggal 1 April,6819984040,-1001360531894,2537813,<NA>,<NA>,0,Terakhir tanggal 1 April,https://t.me/Bibitid/2537821,2025-S1,Saham yang cocok buat pemula gais🙏🏽 RT RT Kok...,saham yang cocok buat pemula gais kok bisa pad...,saham yang cocok buat pemula gais kok bisa pad...
822475,2537823,2025-05-24 21:58:33,met malam Sobits. sampai jumpa besok pagi.,5947896084,-1001360531894,<NA>,<NA>,<NA>,0,met malam Sobits. sampai jumpa besok pagi.,https://t.me/Bibitid/2537823,2025-S1,met malam Sobits. sampai jumpa besok pagi.,met malam sobits sampai jumpa besok pagi,selamat malam sobits sampai jumpa besok pagi


## Analyze data distribution

In [ ]:
import matplotlib.pyplot as plt

original_plt = original_df.groupby(original_df['date'].dt.to_period('Q')).size()
preprocessed_plt = preprocessed_df.groupby(preprocessed_df['date'].dt.to_period('Q')).size()

original_plt.index = original_plt.index.to_timestamp()
preprocessed_plt.index = preprocessed_plt.index.to_timestamp()

plt.figure(figsize=(12, 6))

original_plt.sort_index().plot(label='Original', marker='o')
preprocessed_plt.sort_index().plot(label='Preprocessed', marker='s')

plt.title('Yearly Message Volume Comparison')
plt.xlabel('Year')
plt.ylabel('Number of Messages')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Retrieving language model

In [ ]:
import os
import fasttext
import urllib.request

filename = 'lid.176.bin'
model_url = 'https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin'
model_path = ''

try:
  model_dir = './models/fasttext'
  if not os.path.exists(model_dir):
    print('Creating model directory...')
    os.makedirs(model_dir, exist_ok=True)
  else:
    print('Model directory already exists.')

  model_path = os.path.join(model_dir, filename)

  if not os.path.exists(model_path):
    print('Model not found locally. Downloading...')
    urllib.request.urlretrieve(model_url, model_path)
    print('Download completed!')

  print(f'Loading model from {model_path}...')
  model = fasttext.load_model(model_path)
  print('Model loaded successfully!')

except Exception as e:
  print(f'Error loading FastText model: {e}')

## Identify language

In [ ]:
fasttext_df = preprocessed_df.copy()

texts = fasttext_df['basic_clean'].tolist()

predictions = model.predict(texts)

fasttext_df['lang_detected'] = [label[0].replace('__label__', '') for label in predictions[0]]
fasttext_df['lang_confidence'] = [float(score[0]) for score in predictions[1]]

lang_stats = fasttext_df.groupby('lang_detected')['lang_confidence'].agg(
  count='count',
  min='min',
  q25=lambda x: x.quantile(0.25),
  mean='mean',
  median='median',
  q90=lambda x: x.quantile(0.9),
  max='max'
).reset_index().sort_values(by='count', ascending=False)

display(lang_stats)

## Filter text with Bahasa Indonesia

In [ ]:
fasttext_df = fasttext_df[
  (fasttext_df['lang_detected'] == 'id')
].copy()

display(fasttext_df)

## Analyze data distribution

In [ ]:
import matplotlib.pyplot as plt

original_plt = original_df.groupby(original_df['date'].dt.to_period('Q')).size()
preprocessed_plt = preprocessed_df.groupby(preprocessed_df['date'].dt.to_period('Q')).size()
fasttext_plt = fasttext_df.groupby(fasttext_df['date'].dt.to_period('Q')).size()

original_plt.index = original_plt.index.to_timestamp()
preprocessed_plt.index = preprocessed_plt.index.to_timestamp()
fasttext_plt.index = fasttext_plt.index.to_timestamp()

plt.figure(figsize=(12, 6))

original_plt.sort_index().plot(label='Original', marker='o')
preprocessed_plt.sort_index().plot(label='Preprocessed', marker='s')
fasttext_plt.sort_index().plot(label='Language Filtered', marker='^')

plt.title('Yearly Message Volume Comparison')
plt.xlabel('Year')
plt.ylabel('Number of Messages')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Store language filtered dataframe to local file

In [ ]:
from datetime import datetime

fasttext_df['date'] = fasttext_df['date'].dt.tz_localize(None)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
excel_filename = f'language_filtered_telegram_messages_{timestamp}.xlsx'
fasttext_df.to_excel(excel_filename, index=False)

file_path = f'./data/'
!mkdir -p $file_path
!mv $excel_filename $file_path
print(f'Successfully moved {excel_filename} to {file_path}')

## Train POS tagging

In [ ]:
from flair.datasets import UD_INDONESIAN
from flair.embeddings import WordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

corpus = UD_INDONESIAN()

tag_type = 'upos'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

embedding = WordEmbeddings('id')

tagger = SequenceTagger(
  hidden_size=256,
  embeddings=embedding,
  tag_dictionary=tag_dictionary,
  tag_type=tag_type,
  use_crf=True
)

trainer = ModelTrainer(tagger, corpus)

trainer.train(
  base_path='pos-id-model',
  learning_rate=0.1,
  mini_batch_size=32,
  max_epochs=10
)

## Store POS tagger model

In [ ]:
dir_name = 'pos-id-model'
dir_source = f'./{dir_name}'
dir_target = f'./models/{dir_name}'

!mkdir -p $(dirname $dir_target)

!mv $dir_source $dir_target

print(f'Successfully moved {dir_name} to {dir_target}')

## Retrieving language filtered dataframe from local file

In [ ]:
import os
import pandas as pd

try:
  filename = 'language_filtered_telegram_messages_20250529_185638.xlsx'
  file_path = f'./data/{filename}'

  if not os.path.exists(file_path):
    raise FileNotFoundError(f'File {file_path} does not exist')
  
  print(f'Reading from {file_path} to dataframe')

  fasttext_df = pd.read_excel(file_path)
  
  fasttext_df['date'] = pd.to_datetime(
    fasttext_df['date'],
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
  )

  print(f'Successfully read from {file_path} to dataframe')

except FileNotFoundError:
  fasttext_df = pd.DataFrame()
  print(f'File not found')

except Exception as e:
  fasttext_df = pd.DataFrame()
  print(f'Error reading file: {e}')

display(fasttext_df)

## Predict POS tagging

In [ ]:
from flair.models import SequenceTagger
from flair.data import Sentence
from tqdm import tqdm

texts = fasttext_df['basic_clean'].tolist()

tagger = SequenceTagger.load('models/pos-id-model/best-model.pt')

batch_size = 10000
pos_results = []

for i in tqdm(range(0, len(texts), batch_size)):
  batch_texts = texts[i:i + batch_size]
  batch_sentences = [Sentence(text) for text in batch_texts]

  tagger.predict(batch_sentences)

  for sentence in batch_sentences:
    tags = []
    for token in sentence:
      tag = token.tag if token.labels else None
      tags.append((token.text, tag))
    pos_results.append(tags)

postag_df = fasttext_df.copy()

postag_df['pos_tags'] = pos_results

display(postag_df)

## Filter relevant POS and them stem

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from functools import lru_cache
from pandarallel import pandarallel

pandarallel.initialize()

factory = StemmerFactory()
stemmer = factory.create_stemmer()

@lru_cache(maxsize=10000)
def cached_stem(word):
  return stemmer.stem(word)

allowed_tags = {'PROPN', 'NOUN', 'VERB', 'ADJ', 'ADV'}

def filter_and_stem(tagged_tokens):
  return [
    stemmed for word, tag in tagged_tokens
    if tag in allowed_tags and (stemmed := cached_stem(word))
  ]

postag_df['stemmed_tokens'] = postag_df['pos_tags'].parallel_apply(filter_and_stem)
postag_df = postag_df[postag_df['stemmed_tokens'].str.len() > 0].copy()


## Store POS tagged messsages

In [ ]:
from datetime import datetime

postag_df['date'] = postag_df['date'].dt.tz_localize(None)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
excel_filename = f'pos_tagged_telegram_messages_{timestamp}.xlsx'
postag_df.to_excel(excel_filename, index=False)

file_path = f'./data/'
!mkdir -p $file_path
!mv $excel_filename $file_path
print(f'Successfully moved {excel_filename} to {file_path}')

## Retrieve POS tagged and filtered messages from local file

In [ ]:
import pandas as pd

try:
  filename = 'pos_tagged_telegram_messages_20250529_215008.xlsx'
  file_path = f'./data/{filename}'

  postag_df = pd.read_excel(file_path)

  print(f'Successfully read from {file_path} to dataframe')

except FileNotFoundError:
  postag_df = pd.DataFrame()
  print(f'File not found')

except Exception as e:
  postag_df = pd.DataFrame()
  print(f'Error reading file: {e}')

display(postag_df)

## Run LDA topic modeling

In [ ]:
from gensim import corpora, models
from gensim.models import CoherenceModel
from pandarallel import pandarallel
import ast
import matplotlib.pyplot as plt

pandarallel.initialize()

messages_df = postag_df.copy()

messages_df['stemmed_tokens'] = messages_df['stemmed_tokens'].parallel_apply(
  lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

tokenized_docs = messages_df['stemmed_tokens'].tolist()

dictionary = corpora.Dictionary(tokenized_docs)
dictionary.filter_extremes(no_below=10, no_above=0.8)

corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

filtered = [(i, doc_bow, doc) for i, (doc_bow, doc) in enumerate(zip(corpus, tokenized_docs)) if len(doc_bow) > 0]
indices, corpus, tokenized_docs = zip(*filtered)

def compute_coherence_values(dictionary, corpus, texts, start=10, limit=15, step=1):
  coherence_values = []
  model_list = []
  for num_topics in range(start, limit + 1, step):
    model = models.LdaModel(
      corpus=corpus,
      id2word=dictionary,
      num_topics=num_topics,
      random_state=42,
      passes=10,
      alpha='auto'
    )
    model_list.append(model)
    coherence_model = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_values.append(coherence_model.get_coherence())
  return model_list, coherence_values

start, limit, step = 10, 15, 1
model_list, coherence_values = compute_coherence_values(dictionary, corpus, tokenized_docs, start, limit, step)

## Visualize coherence score

In [ ]:
x = list(range(start, limit + 1, step))
plt.figure(figsize=(10, 6))
plt.plot(x, coherence_values)

for i, (num_topics, score) in enumerate(zip(x, coherence_values)):
  plt.text(num_topics, score, f'{score:.3f}', ha='center', va='bottom', fontsize=9)

plt.xlabel('Number of Topics')
plt.ylabel('Coherence Score (c_v)')
plt.title('Optimal Number of Topics Based on Coherence')
plt.grid(True)
plt.show()

## Attach topic to each message and store

In [ ]:
import os
from datetime import datetime

output_dir = './data/topic_assignments'
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

for num_topics, model in zip(x, model_list):
  print(f'Assigning topics for {num_topics} topics...')

  dominant_topics = []
  topic_keywords = []

  for doc_bow in corpus:
    topic_probs = model.get_document_topics(doc_bow, minimum_probability=0.0)
    dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
    dominant_topics.append(dominant_topic)
    topic_keywords.append(', '.join([word for word, _ in model.show_topic(dominant_topic, topn=10)]))

  assigned_df = messages_df.iloc[list(indices)].copy()
  assigned_df['assigned_topic'] = dominant_topics
  assigned_df['topic_keywords'] = topic_keywords

  print(f'Storing messages with {num_topics} topics...')
  assigned_df.to_excel(
    os.path.join(output_dir, f'{num_topics}_assigned_topics_telegram_messages_{timestamp}.xlsx'), 
    index=False
  )

## Retrieved topic attributed messages from local file

In [ ]:
import pandas as pd

try:
  filename = '11_assigned_topics_telegram_messages_20250530_172416.xlsx'
  file_path = f'./data/topic_assignments/{filename}'

  messages_df = pd.read_excel(file_path)

  print(f'Successfully read from {file_path} to dataframe')

except FileNotFoundError:
  messages_df = pd.DataFrame()
  print(f'File not found')

except Exception as e:
  messages_df = pd.DataFrame()
  print(f'Error reading file: {e}')

display(messages_df)

## List down topics

In [ ]:
unique_topic_pairs = messages_df[['assigned_topic', 'topic_keywords']].drop_duplicates()
unique_topic_pairs = unique_topic_pairs.sort_values(by=['assigned_topic', 'topic_keywords'])
display(unique_topic_pairs)

## Analyze sample messages

In [ ]:
sample_df = (
  messages_df.groupby(['assigned_topic'], group_keys=False)
  .apply(lambda x: x.sample(n=10, random_state=42) if len(x) >= 10 else x)
  .reset_index(drop=True)
)

sample_df = sample_df.sort_values(by=['assigned_topic'])

display(sample_df)

## Visualize topic distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

messages_df['quarter'] = messages_df['date'].dt.to_period('Q').astype(str)

topic_counts = messages_df.groupby(['quarter', 'assigned_topic']).size().reset_index(name='count')

topic_distribution = topic_counts.pivot(index='quarter', columns='assigned_topic', values='count').fillna(0)

topic_distribution = topic_distribution.sort_index()

plt.figure(figsize=(14, 8))
sns.heatmap(topic_distribution.T, cmap='YlGnBu', linewidths=0.5, annot=True, fmt='.0f')
plt.title('Topic Frequency by Quarter')
plt.ylabel('Topic')
plt.xlabel('Quarter')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## Identify knowledge roles

In [25]:
import pandas as pd
import networkx as nx
from datetime import timedelta
from collections import Counter

messages_df['assigned_topic'] = messages_df['assigned_topic'].astype('Int64')

def get_quarter_range(quarter_str, buffer_days=30):
  quarter_start = pd.Period(quarter_str, freq='Q').start_time
  quarter_end = pd.Period(quarter_str, freq='Q').end_time
  return quarter_start - timedelta(days=buffer_days), quarter_end

def classify_user(row, indegree_threshold, betweenness_threshold):
  if row['betweenness'] >= betweenness_threshold:
    return 'indirect_broker'
  elif row['in_degree'] >= indegree_threshold:
    return 'direct_broker'
  elif row['in_degree'] > row['out_degree']:
    return 'seeker'
  elif row['out_degree'] > row['in_degree']:
    return 'contributor'
  else:
    return 'neutral'

role_dist_results = pd.DataFrame()
top_users_results = pd.DataFrame()
all_user_roles_df = pd.DataFrame()
all_topic_edges_df = pd.DataFrame()

quarter_topic_pairs = (
  messages_df.dropna(subset=['assigned_topic'])[['quarter', 'assigned_topic']]
  .drop_duplicates()
  .sort_values(['quarter', 'assigned_topic'])
)

for _, row in quarter_topic_pairs.iterrows():
  quarter = row['quarter']
  topic = row['assigned_topic']
  if pd.isnull(quarter) or pd.isnull(topic):
    continue

  topic_keywords = (
    messages_df[
      (messages_df['quarter'] == quarter) &
      (messages_df['assigned_topic'] == topic)
    ]['topic_keywords']
    .dropna()
    .unique()
  )
  topic_keywords = ', '.join(topic_keywords) if len(topic_keywords) > 0 else ''

  buffered_start, quarter_end = get_quarter_range(quarter)

  subset_messages_df = messages_df[
    (messages_df['date'] >= buffered_start) &
    (messages_df['date'] <= quarter_end) &
    (messages_df['assigned_topic'] == topic)
  ].copy()

  if subset_messages_df.empty:
    continue

  id_to_sender = subset_messages_df.set_index('id')['sender_id'].to_dict()

  quarter_start = pd.Period(quarter, freq='Q').start_time

  reply_msgs_df = subset_messages_df[
    (subset_messages_df['date'] >= quarter_start) &
    (subset_messages_df['date'] <= quarter_end) &
    (subset_messages_df['reply_to_msg_id'].notna())
  ]

  topic_edge_counter = Counter()

  for _, msg in reply_msgs_df.iterrows():
    replier = msg['sender_id']
    reply_to_id = msg['reply_to_msg_id']
    original_sender = id_to_sender.get(reply_to_id)

    if pd.notna(replier) and pd.notna(original_sender) and replier != original_sender:
      key = (original_sender, replier, topic, quarter)
      topic_edge_counter[key] += 1

  G = nx.DiGraph()
  topic_edges = []
  for (original_sender, replier, topic, quarter), weight in topic_edge_counter.items():
    G.add_edge(original_sender, replier, weight=weight)
    topic_edges.append({
      'quarter': quarter,
      'assigned_topic': topic,
      'sender_id': original_sender,
      'replier_id': replier,
      'weight': weight
    })

  all_topic_edges_df = pd.concat([all_topic_edges_df, pd.DataFrame(topic_edges)], ignore_index=True)

  in_deg = dict(G.in_degree())
  out_deg = dict(G.out_degree())
  betweenness = nx.betweenness_centrality(G, normalized=True, weight='weight')

  all_users = list(set(in_deg) | set(out_deg))

  user_roles = pd.DataFrame({
    'sender_id': all_users,
    'in_degree': [in_deg.get(uid, 0) for uid in all_users],
    'out_degree': [out_deg.get(uid, 0) for uid in all_users],
    'betweenness': [betweenness.get(uid, 0) for uid in all_users]
  })

  indegree_threshold = user_roles['in_degree'].quantile(0.95)
  betweenness_threshold = user_roles['betweenness'].quantile(0.95)

  user_roles['role'] = user_roles.apply(
    lambda row: classify_user(row, indegree_threshold, betweenness_threshold), axis=1
  )

  msg_counts = subset_messages_df[
    (subset_messages_df['date'] >= quarter_start) &
    (subset_messages_df['date'] <= quarter_end)
  ].groupby('sender_id').size().reset_index(name='message_count')

  user_roles = user_roles.merge(msg_counts, on='sender_id', how='left').fillna({'message_count': 0})

  user_roles['quarter'] = quarter
  user_roles['assigned_topic'] = topic

  all_user_roles_df = pd.concat([all_user_roles_df, user_roles], ignore_index=True)

  role_counts = user_roles['role'].value_counts().to_dict()

  role_dist = pd.DataFrame([{
    'quarter': quarter,
    'assigned_topic': topic,
    'topic_keywords': topic_keywords,
    'contributor': role_counts.get('contributor', 0),
    'seeker': role_counts.get('seeker', 0),
    'neutral': role_counts.get('neutral', 0),
    'direct_broker': role_counts.get('direct_broker', 0),
    'indirect_broker': role_counts.get('indirect_broker', 0)
  }])
  
  role_dist_results = pd.concat([role_dist_results, role_dist], ignore_index=True)

  top_users = {}

  role_metric = {
    'contributor': 'out_degree',
    'seeker': 'in_degree',
    'direct_broker': 'in_degree',
    'indirect_broker': 'betweenness',
    'neutral': 'message_count'
  }

  for role in ['contributor', 'seeker', 'neutral', 'direct_broker', 'indirect_broker']:
    metric = role_metric[role]
    top = (
      user_roles[user_roles['role'] == role]
      .sort_values(metric, ascending=False)
      .head(3)[['sender_id', metric]]
      .to_dict('records')
    )
    top_users[f'top_{role}s'] = top

  top_users_df = pd.DataFrame([{
    'quarter': quarter,
    'assigned_topic': topic,
    'topic_keywords': topic_keywords,
    **top_users
  }])
  top_users_results = pd.concat([top_users_results, top_users_df], ignore_index=True)

## Role Distribution over Quarters

In [ ]:
import seaborn as sns

melted = role_dist_results.melt(
  id_vars=['quarter', 'assigned_topic'],
  value_vars=['contributor', 'seeker', 'neutral', 'direct_broker', 'indirect_broker'],
  var_name='role',
  value_name='count'
)

plt.figure(figsize=(12, 6))
heatmap_data = melted.pivot_table(index='role', columns='quarter', values='count', aggfunc='sum')
sns.heatmap(heatmap_data, annot=True, fmt='g', cmap='YlGnBu')
plt.title('Role Distribution Across Quarters')
plt.ylabel('Role')
plt.xlabel('Quarter')
plt.tight_layout()
plt.show()

# Role-Topic Distribution over Quarters

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

melted = role_dist_results.melt(
  id_vars=['quarter', 'assigned_topic'],
  value_vars=['contributor', 'seeker', 'neutral', 'direct_broker', 'indirect_broker'],
  var_name='role',
  value_name='count'
)

topics = sorted(melted['assigned_topic'].dropna().unique())
n_cols = 2
n_rows = int(np.ceil(len(topics) / n_cols))

cell_width = 6
cell_height = 4.5
fig, axes = plt.subplots(n_rows, n_cols, figsize=(cell_width * n_cols, cell_height * n_rows))
axes = axes.flatten()

max_count = melted['count'].max()
cmap = sns.color_palette("YlGnBu", as_cmap=True)

for i, topic in enumerate(topics):
  topic_data = melted[melted['assigned_topic'] == topic]
  pivot = topic_data.pivot_table(index='role', columns='quarter', values='count', aggfunc='sum').fillna(0)

  ax = axes[i]
  sns.heatmap(
    pivot,
    ax=ax,
    cmap=cmap,
    vmin=0,
    vmax=max_count,
    linewidths=0.5,
    linecolor='gray',
    annot=True,
    fmt=".0f",
    annot_kws={'size': 8, 'rotation': 90},
    cbar=True
  )

  ax.set_title(f'Topic {topic}', fontsize=12)
  ax.set_xlabel('Quarter', fontsize=10)
  ax.set_ylabel('Role', fontsize=10)
  ax.tick_params(axis='x', labelrotation=90)
  ax.tick_params(axis='y', labelrotation=0)

for j in range(i + 1, len(axes)):
  axes[j].axis('off')

plt.suptitle('Role Distribution by Topic and Quarter', fontsize=18, y=0.96)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


## Decide which topic quarter to visualize

In [ ]:
role_coverage = (
  all_user_roles_df
  .groupby(['quarter', 'assigned_topic', 'role'])
  .size()
  .unstack(fill_value=0)
  .reset_index()
)

complete_role_pairs = role_coverage[
  (role_coverage['contributor'] > 0) &
  (role_coverage['seeker'] > 0) &
  (role_coverage['direct_broker'] > 0) &
  (role_coverage['indirect_broker'] > 0) &
  (role_coverage['neutral'] > 0)
]

user_counts = (
  all_user_roles_df
  .groupby(['quarter', 'assigned_topic'])['sender_id']
  .nunique()
  .reset_index(name='user_count')
)

complete_role_pairs = complete_role_pairs.merge(user_counts, on=['quarter', 'assigned_topic'])

filtered = complete_role_pairs[
  (complete_role_pairs['user_count'] >= 20) & 
  (complete_role_pairs['user_count'] <= 100)
]

filtered = filtered.sort_values(['user_count'])

display(filtered)

## Build graph

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.patches as mpatches

import numpy as np

def radial_layout(G, role_dict):
  pos = {}
  roles_by_layer = {
    0: ['contributor'],
    1: ['direct_broker', 'indirect_broker'],
    2: ['seeker']
  }

  for layer, roles in roles_by_layer.items():
    nodes = [n for n, role in role_dict.items() if role in roles]
    if not nodes:
      continue

    angle_step = 2 * np.pi / len(nodes)
    radius = (layer + 1) * 2.5

    for i, node in enumerate(nodes):
      angle = i * angle_step
      x = radius * np.cos(angle)
      y = radius * np.sin(angle)
      pos[node] = (x, y)

  return pos

selected_quarter = '2022Q3'
selected_topic = 2
quarter_filtered = filtered[
  (filtered['quarter'] == selected_quarter) &
  (filtered['assigned_topic'] == selected_topic)
]
sample = quarter_filtered.iloc[0]
quarter = sample['quarter']
topic = sample['assigned_topic']
print(f'Selected sample: Quarter {quarter}, Topic {topic}')

edges_df = all_topic_edges_df[
  (all_topic_edges_df['quarter'] == quarter) &
  (all_topic_edges_df['assigned_topic'] == topic)
]

roles_df = all_user_roles_df[
  (all_user_roles_df['quarter'] == quarter) &
  (all_user_roles_df['assigned_topic'] == topic) &
  (all_user_roles_df['role'] != 'neutral')
]

G = nx.DiGraph()
for _, row in edges_df.iterrows():
  G.add_edge(row['sender_id'], row['replier_id'], weight=row['weight'])

valid_nodes = set(roles_df['sender_id'])
G = G.subgraph(valid_nodes).copy()

role_dict = roles_df.set_index('sender_id')['role'].to_dict()
nx.set_node_attributes(G, role_dict, 'role')

role_colors = {
  'contributor': 'skyblue',
  'seeker': 'orange',
  'direct_broker': 'green',
  'indirect_broker': 'red'
}

node_colors = [role_colors.get(G.nodes[n].get('role'), 'black') for n in G.nodes]

plt.figure(figsize=(12, 9))

pos = radial_layout(G, role_dict)

nx.draw(
  G, pos,
  with_labels=True,
  node_color=node_colors,
  edge_color='lightgray',
  node_size=800,
  font_size=8
)

legend = [mpatches.Patch(color=c, label=r) for r, c in role_colors.items()]
plt.legend(handles=legend, title='User Role', loc='best')
plt.title(f'Knowledge Exchange Network\nQuarter: {quarter}, Topic: {topic}')
plt.axis('off')
plt.show()
